In [2]:
import pandas as pd
import requests

In [3]:


# API Key from Alpha Vantage
# R4BTXGGSVD9HT6GU
# OZ4VEG5LX8WR3NFK
# K2Y37FKIZBOPQ9EQ
api_key = 'K2Y37FKIZBOPQ9EQ'

# List of company symbols
symbols = ['ACN', 'CTSH', 'IBM', 'INFY', 'WIT', 'BAH']

# Initialize an empty list to store the data
data_list = []

# Loop through each symbol and fetch the annual income statements
for symbol in symbols:
    url_Alpha = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
    response = requests.get(url_Alpha)
    data = response.json()
    
    # Check if there are annual reports and add their data to the list
    if 'annualReports' in data:
        for report in data['annualReports']:
            report['symbol'] = symbol  # Add the company symbol to the report
            data_list.append(report)

# Create a DataFrame from the list of data
df = pd.DataFrame(data_list)



In [11]:
# Reorganicing columns in df

columns = ['symbol'] + [col for col in df.columns if col != 'symbol']
df = df[columns]

In [12]:
# Configure pandas to display a large number of columns
pd.set_option('display.max_columns', None)  # None means no maximum value
pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame repr from wrapping into multiple lines
print(df)

   symbol fiscalDateEnding reportedCurrency  grossProfit totalRevenue costOfRevenue costofGoodsAndServicesSold operatingIncome sellingGeneralAndAdministrative researchAndDevelopment operatingExpenses investmentIncomeNet netInterestIncome interestIncome interestExpense nonInterestIncome otherNonOperatingIncome depreciation depreciationAndAmortization incomeBeforeTax incomeTaxExpense interestAndDebtExpense netIncomeFromContinuingOperations comprehensiveIncomeNetOfTax        ebit      ebitda   netIncome
0     ACN       2023-08-31              USD   8356684000  64111745000   55755061000                43380138000      8809889000                     10858572000             1298657000       13673580000           280409000         -47525000      232884000        47525000        -135586000                96559000    620659000                   453205000      9007359000       2135802000               47525000                        7003530000                  7318798000  9054884000  9508089000 

In [13]:
# DataFrame filter to obtein rows "reportedCurrency" is "INR"
df_inr = df[df['reportedCurrency'] == 'INR']

# unique dates from "fiscalDateEnding"
Unique_dates = df_inr['fiscalDateEnding'].unique().tolist()


In [25]:
# API KEY openexchangerates
api_key = '87bf2801524b4db68c05bb810726d141'

# dates
fechas = Unique_dates

# Initialize an empty list to store the data
exchange_dict = {}

# La URL para la consulta de la API
for fecha in fechas:
    url_Alpha = f'https://openexchangerates.org/api/historical/{fecha}.json?app_id={api_key}&symbols=INR&base=USD'
    response = requests.get(url_Alpha)
    fx = response.json()
    exchange_dict[fecha] = fx['rates']['INR']



In [30]:
df['exchange_rate'] = None
for index, row in df.iterrows():
    # Verify condition:
    if row['reportedCurrency'] == 'INR' and row['fiscalDateEnding'] in exchange_dict:
        # update fx
        df.at[index, 'exchange_rate'] = exchange_dict[row['fiscalDateEnding']]


In [31]:
print(df)

   symbol fiscalDateEnding reportedCurrency  grossProfit totalRevenue costOfRevenue costofGoodsAndServicesSold operatingIncome sellingGeneralAndAdministrative researchAndDevelopment operatingExpenses investmentIncomeNet netInterestIncome interestIncome interestExpense nonInterestIncome otherNonOperatingIncome depreciation depreciationAndAmortization incomeBeforeTax incomeTaxExpense interestAndDebtExpense netIncomeFromContinuingOperations comprehensiveIncomeNetOfTax        ebit      ebitda   netIncome exchange_rate
0     ACN       2023-08-31              USD   8356684000  64111745000   55755061000                43380138000      8809889000                     10858572000             1298657000       13673580000           280409000         -47525000      232884000        47525000        -135586000                96559000    620659000                   453205000      9007359000       2135802000               47525000                        7003530000                  7318798000  905488400

In [32]:
print(exchange_dict)

{'2023-03-31': 82.18065, '2022-03-31': 75.909451, '2021-03-31': 73.203743, '2020-03-31': 75.334994, '2019-03-31': 69.37925, '2018-03-31': 65.0798, '2017-03-31': 64.8067, '2016-03-31': 66.24988, '2015-03-31': 62.39897, '2014-03-31': 59.93978, '2013-03-31': 54.335035, '2012-03-31': 50.707392, '2011-03-31': 44.565151, '2010-03-31': 44.91249, '2009-03-31': 50.767667, '2008-03-31': 40.016241, '2007-03-31': 43.348325, '2006-03-31': 44.538871, '2005-03-31': 43.665684, '2004-03-31': 43.409756}


In [10]:
df.to_excel('consulting_companies.xlsx', index=False)
